In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_predict
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
import time
import shap

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from mlxtend.regressor import StackingRegressor
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

c:\Users\양태영\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
gwangju_preprocessed = pd.read_csv("Gwangju_preprocess.csv")
busan_preprocessed = pd.read_csv("Busan_preprocess.csv")
daegu_preprocessed = pd.read_csv("Daegu_preprocess.csv")
daejeon_preprocessed = pd.read_csv("Daejeon_preprocess.csv")
incheon_preprocessed = pd.read_csv("Incheon_preprocess.csv")
seoul_preprocessed = pd.read_csv("Seoul_preprocess.csv")

In [7]:
gwangju_preprocessed

,Temp,Humi,WS,WD,Solar,Day_sin,Day_cos,T8,T9,T10,...,T13,T14,T15,T16,T17,T18,D1_Temp,D1_Humi,D1_WS,D1_Solar
0,-7.8,83.0,0.0,0.0,0.00,0.017213,0.999852,1,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.00
1,-5.7,74.0,0.0,0.0,0.22,0.017213,0.999852,0,1,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.00
2,-3.2,61.0,3.5,50.0,1.13,0.017213,0.999852,0,0,1,...,0,0,0,0,0,0,0.0,0.0,0.0,0.00
3,-2.7,57.0,1.2,360.0,1.70,0.017213,0.999852,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.00
4,-1.4,55.0,1.6,50.0,2.00,0.017213,0.999852,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40178,-2.5,76.0,1.4,230.0,1.36,0.017213,0.999852,0,0,0,...,0,1,0,0,0,0,-5.0,85.0,1.5,0.34
40179,-1.9,78.0,1.4,230.0,1.29,0.017213,0.999852,0,0,0,...,0,0,1,0,0,0,-5.0,86.0,1.4,0.21
40180,-2.3,86.0,0.5,200.0,0.70,0.017213,0.999852,0,0,0,...,0,0,0,1,0,0,-5.3,83.0,2.3,0.18
40181,-2.4,94.0,1.7,230.0,0.26,0.017213,0.999852,0,0,0,...,0,0,0,0,1,0,-5.8,91.0,1.8,0.11


1.데이터 분할

In [3]:
gwangju_train = gwangju_preprocessed.iloc[:24112, :]
gwangju_val = gwangju_preprocessed.iloc[24112:32142, :]
gwangju_test = gwangju_preprocessed.iloc[32142:, :]
gwangju_train_val = gwangju_preprocessed.iloc[:32142, :]

busan_train = busan_preprocessed.iloc[:24112, :]
busan_val = busan_preprocessed.iloc[24112:32142, :]
busan_test = busan_preprocessed.iloc[32142:, :]
busan_train_val = busan_preprocessed.iloc[:32142, :]

daegu_train = daegu_preprocessed.iloc[:24112, :]
daegu_val = daegu_preprocessed.iloc[24112:32142, :]
daegu_test = daegu_preprocessed.iloc[32142:, :]
daegu_train_val = daegu_preprocessed.iloc[:32142, :]

daejeon_train = daejeon_preprocessed.iloc[:24112, :]
daejeon_val = daejeon_preprocessed.iloc[24112:32142, :]
daejeon_test = daejeon_preprocessed.iloc[32142:, :]
daejeon_train_val = daejeon_preprocessed.iloc[:32142, :]

incheon_train = incheon_preprocessed.iloc[:24112, :]
incheon_val = incheon_preprocessed.iloc[24112:32142, :]
incheon_test = incheon_preprocessed.iloc[32142:, :]
incheon_train_val = incheon_preprocessed.iloc[:32142, :]

seoul_train = seoul_preprocessed.iloc[:24112, :]
seoul_val = seoul_preprocessed.iloc[24112:32142, :]
seoul_test = seoul_preprocessed.iloc[32142:, :]
seoul_train_val = seoul_preprocessed.iloc[:32142, :]

2.모델 입력에 맞게 데이터 변환   (코드 실행시 각 지역을 한작업으로 수행합니다. 총 6작업으로 나누어 진행하였습니다.)

지역 선택

In [6]:
# Gwangju
X_train_rf_gbm = gwangju_train.drop(columns=['Solar'])
y_train_rf_gbm = gwangju_train['Solar']
X_val_rf_gbm = gwangju_val.drop(columns=['Solar'])
y_val_rf_gbm = gwangju_val['Solar']
X_test_rf_gbm = gwangju_test.drop(columns=['Solar'])
y_test_rf_gbm = gwangju_test['Solar']
X_train_val_rf_gbm = gwangju_train_val.drop(columns=['Solar'])
y_train_val_rf_gbm = gwangju_train_val['Solar']

In [4]:
# Busan
X_train_rf_gbm = busan_train.drop(columns=['Solar'])
y_train_rf_gbm = busan_train['Solar']
X_val_rf_gbm = busan_val.drop(columns=['Solar'])
y_val_rf_gbm = busan_val['Solar']
X_test_rf_gbm = busan_test.drop(columns=['Solar'])
y_test_rf_gbm = busan_test['Solar']
X_train_val_rf_gbm = busan_train_val.drop(columns=['Solar'])
y_train_val_rf_gbm = busan_train_val['Solar']

In [4]:
# Daegu
X_train_rf_gbm = daegu_train.drop(columns=['Solar'])
y_train_rf_gbm = daegu_train['Solar']
X_val_rf_gbm = daegu_val.drop(columns=['Solar'])
y_val_rf_gbm = daegu_val['Solar']
X_test_rf_gbm = daegu_test.drop(columns=['Solar'])
y_test_rf_gbm = daegu_test['Solar']
X_train_val_rf_gbm = daegu_train_val.drop(columns=['Solar'])
y_train_val_rf_gbm = daegu_train_val['Solar']

In [4]:
# Daejeon
X_train_rf_gbm = daejeon_train.drop(columns=['Solar'])
y_train_rf_gbm = daejeon_train['Solar']
X_val_rf_gbm = daejeon_val.drop(columns=['Solar'])
y_val_rf_gbm = daejeon_val['Solar']
X_test_rf_gbm = daejeon_test.drop(columns=['Solar'])
y_test_rf_gbm = daejeon_test['Solar']
X_train_val_rf_gbm = daejeon_train_val.drop(columns=['Solar'])
y_train_val_rf_gbm = daejeon_train_val['Solar']

In [4]:
# Incheon
X_train_rf_gbm = incheon_train.drop(columns=['Solar'])
y_train_rf_gbm = incheon_train['Solar']
X_val_rf_gbm = incheon_val.drop(columns=['Solar'])
y_val_rf_gbm = incheon_val['Solar']
X_test_rf_gbm = incheon_test.drop(columns=['Solar'])
y_test_rf_gbm = incheon_test['Solar']
X_train_val_rf_gbm = incheon_train_val.drop(columns=['Solar'])
y_train_val_rf_gbm = incheon_train_val['Solar']

In [4]:
# Seoul
X_train_rf_gbm = seoul_train.drop(columns=['Solar'])
y_train_rf_gbm = seoul_train['Solar']
X_val_rf_gbm = seoul_val.drop(columns=['Solar'])
y_val_rf_gbm = seoul_val['Solar']
X_test_rf_gbm = seoul_test.drop(columns=['Solar'])
y_test_rf_gbm = seoul_test['Solar']
X_train_val_rf_gbm = seoul_train_val.drop(columns=['Solar'])
y_train_val_rf_gbm = seoul_train_val['Solar']

모든 지역 공통

In [5]:
# XGBoost를 위한 데이터 준비 공통
import xgboost as xgb

# 범주형 변수 설정
dtrain = xgb.DMatrix(X_train_rf_gbm, label=y_train_rf_gbm)
dval = xgb.DMatrix(X_val_rf_gbm, label=y_val_rf_gbm)
dtest = xgb.DMatrix(X_test_rf_gbm, label=y_test_rf_gbm)
dtrain_val = xgb.DMatrix(X_train_val_rf_gbm, label=y_train_val_rf_gbm)

# LightGBM을 위한 데이터 준비 공통
import lightgbm as lgb

ltrain = lgb.Dataset(X_train_rf_gbm, label=y_train_rf_gbm)
lval = lgb.Dataset(X_val_rf_gbm, label=y_val_rf_gbm)
ltest = lgb.Dataset(X_test_rf_gbm, label=y_test_rf_gbm)
ltrain_val = lgb.Dataset(X_train_val_rf_gbm, label=y_train_val_rf_gbm)

# CatBoost를 위한 데이터 준비 공통
import catboost as cb

ctrain = cb.Pool(data=X_train_rf_gbm, label=y_train_rf_gbm)
cval = cb.Pool(data=X_val_rf_gbm, label=y_val_rf_gbm)
ctest = cb.Pool(data=X_test_rf_gbm, label=y_test_rf_gbm)
ctrain_val = cb.Pool(data=X_train_val_rf_gbm, label=y_train_val_rf_gbm)

In [10]:
pip install optuna

     -------------------------------------- 364.4/364.4 KB 2.8 MB/s eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 3.8 MB/s eta 0:00:00
     -------------------------------------- 233.5/233.5 KB 2.4 MB/s eta 0:00:00
     -------------------------------------- 161.8/161.8 KB 3.3 MB/s eta 0:00:00
     ---------------------------------------- 78.6/78.6 KB 2.2 MB/s eta 0:00:00
     -------------------------------------- 298.4/298.4 KB 1.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\양태영\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
# optuna + seed 고정 / train으로 학습 val을 통해 평가

import optuna
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Set random seed for reproducibility
SEED = 42  # 랜덤 시드 설정 추가

# Random Forest Objective
def objective_rf(trial):
    
    np.random.seed(SEED)  # 랜덤 시드 설정 추가

    
    params = {
        'n_estimators': trial.suggest_categorical('n_estimators', [128]),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2'])
    }
    model = RandomForestRegressor(**params, random_state=SEED)
    model.fit(X_train_rf_gbm, y_train_rf_gbm)
    predictions = model.predict(X_val_rf_gbm)
    rmse = np.sqrt(mean_squared_error(y_val_rf_gbm, predictions))
    return rmse

# Gradient Boosting Objective
def objective_gbm(trial):
    
    np.random.seed(SEED)
    
    params = {
        'n_estimators': trial.suggest_categorical('n_estimators', [100, 250, 500]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.01, 0.05, 0.1]),
        'max_depth': trial.suggest_categorical('max_depth', [5, 10]),
        'loss': trial.suggest_categorical('loss', ['quantile', 'huber'])
    }
    model = GradientBoostingRegressor(**params, random_state=SEED)
    model.fit(X_train_rf_gbm, y_train_rf_gbm)
    predictions = model.predict(X_val_rf_gbm)
    rmse = np.sqrt(mean_squared_error(y_val_rf_gbm, predictions))
    return rmse

# XGBoost Objective for Optuna
def objective_xgb(trial):
    # Hyperparameter suggestion
    np.random.seed(SEED)

    params = {
        'objective': 'reg:squarederror',
        'learning_rate': trial.suggest_categorical('learning_rate', [0.01, 0.05, 0.1]),
        'max_depth': trial.suggest_categorical('max_depth', [6, 8, 10]),
        'subsample': trial.suggest_categorical('subsample', [0.5, 0.75, 1.0]),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.5, 0.75, 1.0]),
        'booster': trial.suggest_categorical('booster', ['gbtree', 'dart']),
        'random_state': SEED  # XGBoost의 랜덤 시드 설정 추가
    }
    # Separate num_boost_round from params
    num_boost_round = trial.suggest_categorical('n_estimators', [250, 500, 1000])

    # XGBoost Training with DMatrix
    model = xgb.train(params, dtrain, num_boost_round=num_boost_round)

    # Predict on test set
    predictions = model.predict(dval)

    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(dval.get_label(), predictions))
    return rmse

# LightGBM Objective
def objective_lgb(trial):
    
    np.random.seed(SEED)
    
    params = {
        'objective': 'regression',
        'learning_rate': trial.suggest_categorical('learning_rate', [0.01, 0.05, 0.1]),
        'num_leaves': trial.suggest_int('num_leaves', 64, 64),
        'subsample': trial.suggest_categorical('subsample', [0.5]),
        'feature_fraction': trial.suggest_categorical('feature_fraction', [1.0]),
        'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'dart']),
        'seed': SEED  # LightGBM의 랜덤 시드 설정 추가
    }
    num_boost_round = trial.suggest_categorical('n_estimators', [1000, 1500])
    model = lgb.train(params, ltrain, num_boost_round=num_boost_round)
    predictions = model.predict(X_val_rf_gbm)
    rmse = np.sqrt(mean_squared_error(y_val_rf_gbm, predictions))
    return rmse

# CatBoost Objective
def objective_cat(trial):
    
    np.random.seed(SEED)
    
    params = {
        'learning_rate': trial.suggest_categorical('learning_rate', [0.03, 0.1]),
        'depth': trial.suggest_categorical('depth', [4, 6, 10]),
        'l2_leaf_reg': trial.suggest_categorical('l2_leaf_reg', [1, 3, 5, 7, 9]),
        'random_state': SEED
    }
    model = cb.CatBoostRegressor(**params,verbose=0)
    model.fit(ctrain)
    predictions = model.predict(cval)
    rmse = np.sqrt(mean_squared_error(y_val_rf_gbm, predictions))
    return rmse

3. Optuna 하이퍼파라미터 최적화(train : val)

In [6]:
# optuna + seed 고정 / train으로 학습 val을 통해 평가

import optuna
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Set random seed for reproducibility
SEED = 42  # 랜덤 시드 설정 추가

# Random Forest Objective
def objective_rf(trial):
    
    np.random.seed(SEED)  # 랜덤 시드 설정 추가

    
    params = {
        'n_estimators': trial.suggest_categorical('n_estimators', [128]),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2'])
    }
    model = RandomForestRegressor(**params, random_state=SEED)
    model.fit(X_train_rf_gbm, y_train_rf_gbm)
    predictions = model.predict(X_val_rf_gbm)
    rmse = np.sqrt(mean_squared_error(y_val_rf_gbm, predictions))
    return rmse

# Gradient Boosting Objective
def objective_gbm(trial):
    
    np.random.seed(SEED)
    
    params = {
        'n_estimators': trial.suggest_categorical('n_estimators', [100, 250, 500]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.01, 0.05, 0.1]),
        'max_depth': trial.suggest_categorical('max_depth', [5, 10]),
        'loss': trial.suggest_categorical('loss', ['quantile', 'huber'])
    }
    model = GradientBoostingRegressor(**params, random_state=SEED)
    model.fit(X_train_rf_gbm, y_train_rf_gbm)
    predictions = model.predict(X_val_rf_gbm)
    rmse = np.sqrt(mean_squared_error(y_val_rf_gbm, predictions))
    return rmse

# XGBoost Objective for Optuna
def objective_xgb(trial):
    # Hyperparameter suggestion
    np.random.seed(SEED)

    params = {
        'objective': 'reg:squarederror',
        'learning_rate': trial.suggest_categorical('learning_rate', [0.01, 0.05, 0.1]),
        'max_depth': trial.suggest_categorical('max_depth', [6, 8, 10]),
        'subsample': trial.suggest_categorical('subsample', [0.5, 0.75, 1.0]),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.5, 0.75, 1.0]),
        'booster': trial.suggest_categorical('booster', ['gbtree', 'dart']),
        'random_state': SEED  # XGBoost의 랜덤 시드 설정 추가
    }
    # Separate num_boost_round from params
    num_boost_round = trial.suggest_categorical('n_estimators', [250, 500, 1000])

    # XGBoost Training with DMatrix
    model = xgb.train(params, dtrain, num_boost_round=num_boost_round)

    # Predict on test set
    predictions = model.predict(dval)

    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(dval.get_label(), predictions))
    return rmse

# LightGBM Objective
def objective_lgb(trial):
    
    np.random.seed(SEED)
    
    params = {
        'objective': 'regression',
        'learning_rate': trial.suggest_categorical('learning_rate', [0.01, 0.05, 0.1]),
        'num_leaves': trial.suggest_int('num_leaves', 64, 64),
        'subsample': trial.suggest_categorical('subsample', [0.5]),
        'feature_fraction': trial.suggest_categorical('feature_fraction', [1.0]),
        'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'dart']),
        'seed': SEED  # LightGBM의 랜덤 시드 설정 추가
    }
    num_boost_round = trial.suggest_categorical('n_estimators', [1000, 1500])
    model = lgb.train(params, ltrain, num_boost_round=num_boost_round)
    predictions = model.predict(X_val_rf_gbm)
    rmse = np.sqrt(mean_squared_error(y_val_rf_gbm, predictions))
    return rmse

# CatBoost Objective
def objective_cat(trial):
    
    np.random.seed(SEED)
    
    params = {
        'learning_rate': trial.suggest_categorical('learning_rate', [0.03, 0.1]),
        'depth': trial.suggest_categorical('depth', [4, 6, 10]),
        'l2_leaf_reg': trial.suggest_categorical('l2_leaf_reg', [1, 3, 5, 7, 9]),
        'random_state': SEED
    }
    model = cb.CatBoostRegressor(**params,verbose=0)
    model.fit(ctrain)
    predictions = model.predict(cval)
    rmse = np.sqrt(mean_squared_error(y_val_rf_gbm, predictions))
    return rmse

In [7]:
study_rf = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
study_rf.optimize(objective_rf, n_trials=100)

[I 2024-12-02 16:48:10,247] A new study created in memory with name: no-name-793010a5-7c61-4c47-8b92-c490ee6135e3
[I 2024-12-02 16:48:22,499] Trial 0 finished with value: 0.3892836685749838 and parameters: {'n_estimators': 128, 'max_features': 'log2'}. Best is trial 0 with value: 0.3892836685749838.
[I 2024-12-02 16:48:34,312] Trial 1 finished with value: 0.3892836685749838 and parameters: {'n_estimators': 128, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.3892836685749838.
[I 2024-12-02 16:48:45,825] Trial 2 finished with value: 0.3892836685749838 and parameters: {'n_estimators': 128, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.3892836685749838.
[I 2024-12-02 16:48:58,011] Trial 3 finished with value: 0.3892836685749838 and parameters: {'n_estimators': 128, 'max_features': 'log2'}. Best is trial 0 with value: 0.3892836685749838.
[I 2024-12-02 16:49:09,520] Trial 4 finished with value: 0.3892836685749838 and parameters: {'n_estimators': 128, 'max_features': 'log2'}.

In [8]:
study_gbm = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
study_gbm.optimize(objective_gbm, n_trials=100)

[I 2024-12-02 17:04:43,926] A new study created in memory with name: no-name-73d6e8f3-86cc-44d2-971b-6be6bd80afb4


[I 2024-12-02 17:06:34,429] Trial 0 finished with value: 0.39914551467349274 and parameters: {'n_estimators': 250, 'learning_rate': 0.01, 'max_depth': 10, 'loss': 'huber'}. Best is trial 0 with value: 0.39914551467349274.
[I 2024-12-02 17:07:39,567] Trial 1 finished with value: 0.657040855725531 and parameters: {'n_estimators': 250, 'learning_rate': 0.01, 'max_depth': 10, 'loss': 'quantile'}. Best is trial 0 with value: 0.39914551467349274.
[I 2024-12-02 17:08:01,995] Trial 2 finished with value: 0.4672809411911528 and parameters: {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 10, 'loss': 'quantile'}. Best is trial 0 with value: 0.39914551467349274.
[I 2024-12-02 17:08:11,531] Trial 3 finished with value: 0.5703221532675726 and parameters: {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 5, 'loss': 'quantile'}. Best is trial 0 with value: 0.39914551467349274.
[I 2024-12-02 17:09:18,168] Trial 4 finished with value: 0.657040855725531 and parameters: {'n_estimators': 2

In [9]:
study_xgb = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
study_xgb.optimize(objective_xgb, n_trials=100)

[I 2024-12-02 18:31:49,604] A new study created in memory with name: no-name-158cf602-972d-4d6c-bb67-4716ccbea5e1


[I 2024-12-02 18:31:54,674] Trial 0 finished with value: 0.37454453110694885 and parameters: {'learning_rate': 0.05, 'max_depth': 6, 'subsample': 0.75, 'colsample_bytree': 1.0, 'booster': 'gbtree', 'n_estimators': 1000}. Best is trial 0 with value: 0.37454453110694885.
[I 2024-12-02 18:40:15,855] Trial 1 finished with value: 0.3694452941417694 and parameters: {'learning_rate': 0.01, 'max_depth': 6, 'subsample': 1.0, 'colsample_bytree': 1.0, 'booster': 'dart', 'n_estimators': 1000}. Best is trial 1 with value: 0.3694452941417694.
[I 2024-12-02 18:49:24,850] Trial 2 finished with value: 0.37398219108581543 and parameters: {'learning_rate': 0.01, 'max_depth': 8, 'subsample': 0.75, 'colsample_bytree': 0.5, 'booster': 'dart', 'n_estimators': 1000}. Best is trial 1 with value: 0.3694452941417694.
[I 2024-12-02 18:49:32,659] Trial 3 finished with value: 0.3787497282028198 and parameters: {'learning_rate': 0.05, 'max_depth': 8, 'subsample': 1.0, 'colsample_bytree': 1.0, 'booster': 'gbtree', 'n

In [10]:
study_lgb = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
study_lgb.optimize(objective_lgb, n_trials=100)

[I 2024-12-02 20:38:18,517] A new study created in memory with name: no-name-966136c1-2603-40af-8862-c14c6c677f9b


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001333 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1840
[LightGBM] [Info] Number of data points in the train set: 24112, number of used features: 21
[LightGBM] [Info] Start training from score 1.093214


[I 2024-12-02 20:38:23,705] Trial 0 finished with value: 0.376072387725667 and parameters: {'learning_rate': 0.05, 'num_leaves': 64, 'subsample': 0.5, 'feature_fraction': 1.0, 'boosting_type': 'gbdt', 'n_estimators': 1000}. Best is trial 0 with value: 0.376072387725667.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000425 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1840
[LightGBM] [Info] Number of data points in the train set: 24112, number of used features: 21
[LightGBM] [Info] Start training from score 1.093214


[I 2024-12-02 20:38:56,848] Trial 1 finished with value: 0.46839311734161215 and parameters: {'learning_rate': 0.01, 'num_leaves': 64, 'subsample': 0.5, 'feature_fraction': 1.0, 'boosting_type': 'dart', 'n_estimators': 1000}. Best is trial 0 with value: 0.376072387725667.


...

[I 2024-12-02 20:54:34,030] Trial 97 finished with value: 0.366419897466168 and parameters: {'learning_rate': 0.01, 'num_leaves': 64, 'subsample': 0.5, 'feature_fraction': 1.0, 'boosting_type': 'gbdt', 'n_estimators': 1000}. Best is trial 10 with value: 0.366419897466168.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002342 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1840
[LightGBM] [Info] Number of data points in the train set: 24112, number of used features: 21
[LightGBM] [Info] Start training from score 1.093214


[I 2024-12-02 20:54:39,165] Trial 98 finished with value: 0.366419897466168 and parameters: {'learning_rate': 0.01, 'num_leaves': 64, 'subsample': 0.5, 'feature_fraction': 1.0, 'boosting_type': 'gbdt', 'n_estimators': 1000}. Best is trial 10 with value: 0.366419897466168.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001909 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1840
[LightGBM] [Info] Number of data points in the train set: 24112, number of used features: 21
[LightGBM] [Info] Start training from score 1.093214


[I 2024-12-02 20:54:46,036] Trial 99 finished with value: 0.3672010780843957 and parameters: {'learning_rate': 0.01, 'num_leaves': 64, 'subsample': 0.5, 'feature_fraction': 1.0, 'boosting_type': 'gbdt', 'n_estimators': 1500}. Best is trial 10 with value: 0.366419897466168.


In [11]:
study_cat = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
study_cat.optimize(objective_cat, n_trials=100)

[I 2024-12-02 20:54:46,074] A new study created in memory with name: no-name-540e960a-b2b6-4367-aeac-c081d9752fa4


[I 2024-12-02 20:54:51,594] Trial 0 finished with value: 0.37397394515760507 and parameters: {'learning_rate': 0.1, 'depth': 4, 'l2_leaf_reg': 5}. Best is trial 0 with value: 0.37397394515760507.
[I 2024-12-02 20:54:56,861] Trial 1 finished with value: 0.37397394515760507 and parameters: {'learning_rate': 0.1, 'depth': 4, 'l2_leaf_reg': 5}. Best is trial 0 with value: 0.37397394515760507.
[I 2024-12-02 20:55:40,380] Trial 2 finished with value: 0.36795421747982127 and parameters: {'learning_rate': 0.03, 'depth': 10, 'l2_leaf_reg': 1}. Best is trial 2 with value: 0.36795421747982127.
[I 2024-12-02 20:56:23,681] Trial 3 finished with value: 0.36795421747982127 and parameters: {'learning_rate': 0.03, 'depth': 10, 'l2_leaf_reg': 1}. Best is trial 2 with value: 0.36795421747982127.
[I 2024-12-02 20:56:30,863] Trial 4 finished with value: 0.37397341895366776 and parameters: {'learning_rate': 0.1, 'depth': 6, 'l2_leaf_reg': 1}. Best is trial 2 with value: 0.36795421747982127.
[I 2024-12-02 20

4. 모델 학습(데이터 train + val : test)

광주

In [ ]:

# Random Forest
best_params_rf = study_rf.best_params
print("Best parameters for Random Forest:", best_params_rf)

# 모델 학습 및 저장
start_time = time.time()
final_model_rf = RandomForestRegressor(**best_params_rf, random_state=SEED)
final_model_rf.fit(X_train_val_rf_gbm, y_train_val_rf_gbm)
training_time_rf = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_rf, "./gwangju_model/gwangju_rf_opt.pkl")

# 모델 성능 평가
final_predictions_rf = final_model_rf.predict(X_test_rf_gbm)
final_rmse_rf = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_rf))
final_mse_rf = mean_squared_error(y_test_rf_gbm, final_predictions_rf)
final_mae_rf = mean_absolute_error(y_test_rf_gbm, final_predictions_rf)
final_r2_rf = r2_score(y_test_rf_gbm, final_predictions_rf)

print(f"Random Forest - Training Time: {training_time_rf:.2f}s, RMSE: {final_rmse_rf:.4f}, MSE: {final_mse_rf:.4f}, MAE: {final_mae_rf:.4f}, R2: {final_r2_rf:.4f}")

Best parameters for Random Forest: {'n_estimators': 128, 'max_features': 'log2'}
Random Forest - Training Time: 12.96s, RMSE: 0.4227, MSE: 0.1787, MAE: 0.3185, R2: 0.8100

In [15]:
# Gradient Boosting
best_params_gbm = study_gbm.best_params
print("Best parameters for Gradient Boosting:", best_params_gbm)

start_time = time.time()
final_model_gbm = GradientBoostingRegressor(**best_params_gbm)
final_model_gbm.fit(X_train_val_rf_gbm, y_train_val_rf_gbm)
training_time_gbm = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_gbm, "./gwangju_model/gwangju_gbm_opt.pkl")

final_predictions_gbm = final_model_gbm.predict(X_test_rf_gbm)
final_rmse_gbm = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_gbm))
final_mse_gbm = mean_squared_error(y_test_rf_gbm, final_predictions_gbm)
final_mae_gbm = mean_absolute_error(y_test_rf_gbm, final_predictions_gbm)
final_r2_gbm = r2_score(y_test_rf_gbm, final_predictions_gbm)
print(f"Gradient Boosting - Training Time: {training_time_gbm:.2f}s, RMSE: {final_rmse_gbm:.4f}, MSE: {final_mse_gbm:.4f}, MAE: {final_mae_gbm:.4f}, R2: {final_r2_gbm:.4f}")

Best parameters for Gradient Boosting: {'n_estimators': 500, 'learning_rate': 0.05, 'max_depth': 5, 'loss': 'huber'}
Gradient Boosting - Training Time: 66.85s, RMSE: 0.4019, MSE: 0.1615, MAE: 0.2958, R2: 0.8282


In [17]:
# XGBoost
best_params_xgb = study_xgb.best_params
print("Best parameters for XGBoost:", best_params_xgb)

best_num_boost_round = best_params_xgb.pop('n_estimators')
start_time = time.time()
final_model_xgb = xgb.train(best_params_xgb, dtrain_val, num_boost_round=best_num_boost_round)
training_time_xgb = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_xgb, "./gwangju_model/gwangju_xgb_opt.pkl")

# 예측 (test 데이터로 예측)
final_predictions_xgb = final_model_xgb.predict(dtest)

# 평가 지표 계산
final_rmse_xgb = np.sqrt(mean_squared_error(dtest.get_label(), final_predictions_xgb))
final_mse_xgb = mean_squared_error(dtest.get_label(), final_predictions_xgb)
final_mae_xgb = mean_absolute_error(dtest.get_label(), final_predictions_xgb)
final_r2_xgb = r2_score(dtest.get_label(), final_predictions_xgb)

# 결과 출력
print(f"XGBoost - Training Time: {training_time_xgb:.2f}s, RMSE: {final_rmse_xgb:.4f}, MSE: {final_mse_xgb:.4f}, MAE: {final_mae_xgb:.4f}, R2: {final_r2_xgb:.4f}")

Best parameters for XGBoost: {'learning_rate': 0.01, 'max_depth': 8, 'subsample': 0.75, 'colsample_bytree': 0.75, 'booster': 'gbtree', 'n_estimators': 1000}
XGBoost - Training Time: 9.11s, RMSE: 0.4022, MSE: 0.1618, MAE: 0.2982, R2: 0.8280


In [19]:
# LightGBM
best_params_lgb = study_lgb.best_params
print("Best parameters for LightGBM:", best_params_lgb)

start_time = time.time()
final_model_lgb = lgb.train(best_params_lgb, ltrain_val, num_boost_round=best_params_lgb['n_estimators'])
training_time_lgb = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_lgb, "./gwangju_model/gwangju_lgb_opt.pkl")

# 예측 (test 데이터로 예측)
final_predictions_lgb = final_model_lgb.predict(X_test_rf_gbm)  # X_test_rf_gbm 사용

# 평가 지표 계산
final_rmse_lgb = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_lgb))
final_mse_lgb = mean_squared_error(y_test_rf_gbm, final_predictions_lgb)
final_mae_lgb = mean_absolute_error(y_test_rf_gbm, final_predictions_lgb)
final_r2_lgb = r2_score(y_test_rf_gbm, final_predictions_lgb)

# 결과 출력
print(f"LightGBM - Training Time: {training_time_lgb:.2f}s, RMSE: {final_rmse_lgb:.4f}, MSE: {final_mse_lgb:.4f}, MAE: {final_mae_lgb:.4f}, R2: {final_r2_lgb:.4f}")

c:\Users\양태영\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Best parameters for LightGBM: {'learning_rate': 0.01, 'num_leaves': 64, 'subsample': 0.5, 'feature_fraction': 1.0, 'boosting_type': 'gbdt', 'n_estimators': 1500}
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000556 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1828
[LightGBM] [Info] Number of data points in the train set: 32142, number of used features: 21
[LightGBM] [Info] Start training from score 1.267165
LightGBM - Training Time: 8.69s, RMSE: 0.4017, MSE: 0.1614, MAE: 0.2970, R2: 0.8284


In [21]:
# CatBoost
best_params_cat = study_cat.best_params
print("Best parameters for CatBoost:", best_params_cat)

start_time = time.time()
final_model_cat = cb.CatBoostRegressor(**best_params_cat, verbose=0, random_seed=SEED)
final_model_cat.fit(ctrain_val)
training_time_cat = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_cat, "./gwangju_model/gwangju_cat_opt.pkl")

final_predictions_cat = final_model_cat.predict(ctest)
final_rmse_cat = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_cat))
final_mse_cat = mean_squared_error(y_test_rf_gbm, final_predictions_cat)
final_mae_cat = mean_absolute_error(y_test_rf_gbm, final_predictions_cat)
final_r2_cat = r2_score(y_test_rf_gbm, final_predictions_cat)
print(f"CatBoost - Training Time: {training_time_cat:.2f}s, RMSE: {final_rmse_cat:.4f}, MSE: {final_mse_cat:.4f}, MAE: {final_mae_cat:.4f}, R2: {final_r2_cat:.4f}")

Best parameters for CatBoost: {'learning_rate': 0.03, 'depth': 10, 'l2_leaf_reg': 3}
CatBoost - Training Time: 38.93s, RMSE: 0.3983, MSE: 0.1587, MAE: 0.2954, R2: 0.8313


부산


In [9]:
# Random Forest
best_params_rf = study_rf.best_params
print("Best parameters for Random Forest:", best_params_rf)

# 모델 학습 및 저장
start_time = time.time()
final_model_rf = RandomForestRegressor(**best_params_rf, random_state=SEED)
final_model_rf.fit(X_train_val_rf_gbm, y_train_val_rf_gbm)
training_time_rf = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_rf, "./busan_model/busan_rf_opt.pkl")

# 모델 성능 평가
final_predictions_rf = final_model_rf.predict(X_test_rf_gbm)
final_rmse_rf = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_rf))
final_mse_rf = mean_squared_error(y_test_rf_gbm, final_predictions_rf)
final_mae_rf = mean_absolute_error(y_test_rf_gbm, final_predictions_rf)
final_r2_rf = r2_score(y_test_rf_gbm, final_predictions_rf)

print(f"Random Forest - Training Time: {training_time_rf:.2f}s, RMSE: {final_rmse_rf:.4f}, MSE: {final_mse_rf:.4f}, MAE: {final_mae_rf:.4f}, R2: {final_r2_rf:.4f}")

Best parameters for Random Forest: {'n_estimators': 128, 'max_features': 'log2'}
Random Forest - Training Time: 12.84s, RMSE: 0.4313, MSE: 0.1861, MAE: 0.3223, R2: 0.8177


In [10]:
# Gradient Boosting
best_params_gbm = study_gbm.best_params
print("Best parameters for Gradient Boosting:", best_params_gbm)

start_time = time.time()
final_model_gbm = GradientBoostingRegressor(**best_params_gbm)
final_model_gbm.fit(X_train_val_rf_gbm, y_train_val_rf_gbm)
training_time_gbm = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_gbm, "./busan_model/busan_gbm_opt.pkl")

final_predictions_gbm = final_model_gbm.predict(X_test_rf_gbm)
final_rmse_gbm = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_gbm))
final_mse_gbm = mean_squared_error(y_test_rf_gbm, final_predictions_gbm)
final_mae_gbm = mean_absolute_error(y_test_rf_gbm, final_predictions_gbm)
final_r2_gbm = r2_score(y_test_rf_gbm, final_predictions_gbm)
print(f"Gradient Boosting - Training Time: {training_time_gbm:.2f}s, RMSE: {final_rmse_gbm:.4f}, MSE: {final_mse_gbm:.4f}, MAE: {final_mae_gbm:.4f}, R2: {final_r2_gbm:.4f}")

Best parameters for Gradient Boosting: {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 10, 'loss': 'huber'}
Gradient Boosting - Training Time: 46.94s, RMSE: 0.4188, MSE: 0.1754, MAE: 0.3010, R2: 0.8282


In [14]:
# XGBoost
best_params_xgb = study_xgb.best_params
print("Best parameters for XGBoost:", best_params_xgb)

best_num_boost_round = best_params_xgb.pop('n_estimators')
start_time = time.time()
final_model_xgb = xgb.train(best_params_xgb, dtrain_val, num_boost_round=best_num_boost_round)
training_time_xgb = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_xgb, "./busan_model/busan_xgb_opt.pkl")

# 예측 (test 데이터로 예측)
final_predictions_xgb = final_model_xgb.predict(dtest)

# 평가 지표 계산
final_rmse_xgb = np.sqrt(mean_squared_error(dtest.get_label(), final_predictions_xgb))
final_mse_xgb = mean_squared_error(dtest.get_label(), final_predictions_xgb)
final_mae_xgb = mean_absolute_error(dtest.get_label(), final_predictions_xgb)
final_r2_xgb = r2_score(dtest.get_label(), final_predictions_xgb)

# 결과 출력
print(f"XGBoost - Training Time: {training_time_xgb:.2f}s, RMSE: {final_rmse_xgb:.4f}, MSE: {final_mse_xgb:.4f}, MAE: {final_mae_xgb:.4f}, R2: {final_r2_xgb:.4f}")

Best parameters for XGBoost: {'learning_rate': 0.01, 'max_depth': 10, 'subsample': 0.5, 'colsample_bytree': 1.0, 'booster': 'gbtree', 'n_estimators': 500}
XGBoost - Training Time: 9.77s, RMSE: 0.4093, MSE: 0.1676, MAE: 0.2990, R2: 0.8358


In [15]:
# LightGBM
best_params_lgb = study_lgb.best_params
print("Best parameters for LightGBM:", best_params_lgb)

start_time = time.time()
final_model_lgb = lgb.train(best_params_lgb, ltrain_val, num_boost_round=best_params_lgb['n_estimators'])
training_time_lgb = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_lgb, "./busan_model/busan_lgb_opt.pkl")

# 예측 (test 데이터로 예측)
final_predictions_lgb = final_model_lgb.predict(X_test_rf_gbm)  # X_test_rf_gbm 사용

# 평가 지표 계산
final_rmse_lgb = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_lgb))
final_mse_lgb = mean_squared_error(y_test_rf_gbm, final_predictions_lgb)
final_mae_lgb = mean_absolute_error(y_test_rf_gbm, final_predictions_lgb)
final_r2_lgb = r2_score(y_test_rf_gbm, final_predictions_lgb)

# 결과 출력
print(f"LightGBM - Training Time: {training_time_lgb:.2f}s, RMSE: {final_rmse_lgb:.4f}, MSE: {final_mse_lgb:.4f}, MAE: {final_mae_lgb:.4f}, R2: {final_r2_lgb:.4f}")

Best parameters for LightGBM: {'learning_rate': 0.01, 'num_leaves': 64, 'subsample': 0.5, 'feature_fraction': 1.0, 'boosting_type': 'gbdt', 'n_estimators': 1000}
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002858 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2253
[LightGBM] [Info] Number of data points in the train set: 32142, number of used features: 21
[LightGBM] [Info] Start training from score 1.253945


c:\Users\양태영\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


LightGBM - Training Time: 5.86s, RMSE: 0.4072, MSE: 0.1658, MAE: 0.2976, R2: 0.8375


In [16]:
# CatBoost
best_params_cat = study_cat.best_params
print("Best parameters for CatBoost:", best_params_cat)

start_time = time.time()
final_model_cat = cb.CatBoostRegressor(**best_params_cat, verbose=0, random_seed=SEED)
final_model_cat.fit(ctrain_val)
training_time_cat = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_cat, "./busan_model/busan_cat_opt.pkl")

final_predictions_cat = final_model_cat.predict(ctest)
final_rmse_cat = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_cat))
final_mse_cat = mean_squared_error(y_test_rf_gbm, final_predictions_cat)
final_mae_cat = mean_absolute_error(y_test_rf_gbm, final_predictions_cat)
final_r2_cat = r2_score(y_test_rf_gbm, final_predictions_cat)
print(f"CatBoost - Training Time: {training_time_cat:.2f}s, RMSE: {final_rmse_cat:.4f}, MSE: {final_mse_cat:.4f}, MAE: {final_mae_cat:.4f}, R2: {final_r2_cat:.4f}")

Best parameters for CatBoost: {'learning_rate': 0.03, 'depth': 10, 'l2_leaf_reg': 5}
CatBoost - Training Time: 50.34s, RMSE: 0.4072, MSE: 0.1658, MAE: 0.3008, R2: 0.8376


대구

In [12]:
# Random Forest
best_params_rf = study_rf.best_params
print("Best parameters for Random Forest:", best_params_rf)

# 모델 학습 및 저장
start_time = time.time()
final_model_rf = RandomForestRegressor(**best_params_rf, random_state=SEED)
final_model_rf.fit(X_train_val_rf_gbm, y_train_val_rf_gbm)
training_time_rf = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_rf, "./daegu_model/daegu_rf_opt.pkl")

# 모델 성능 평가
final_predictions_rf = final_model_rf.predict(X_test_rf_gbm)
final_rmse_rf = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_rf))
final_mse_rf = mean_squared_error(y_test_rf_gbm, final_predictions_rf)
final_mae_rf = mean_absolute_error(y_test_rf_gbm, final_predictions_rf)
final_r2_rf = r2_score(y_test_rf_gbm, final_predictions_rf)

print(f"Random Forest - Training Time: {training_time_rf:.2f}s, RMSE: {final_rmse_rf:.4f}, MSE: {final_mse_rf:.4f}, MAE: {final_mae_rf:.4f}, R2: {final_r2_rf:.4f}")

Best parameters for Random Forest: {'n_estimators': 128, 'max_features': 'log2'}
Random Forest - Training Time: 12.81s, RMSE: 0.3806, MSE: 0.1448, MAE: 0.2757, R2: 0.8341


In [13]:
# Gradient Boosting
best_params_gbm = study_gbm.best_params
print("Best parameters for Gradient Boosting:", best_params_gbm)

start_time = time.time()
final_model_gbm = GradientBoostingRegressor(**best_params_gbm)
final_model_gbm.fit(X_train_val_rf_gbm, y_train_val_rf_gbm)
training_time_gbm = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_gbm, "./daegu_model/daegu_gbm_opt.pkl")

final_predictions_gbm = final_model_gbm.predict(X_test_rf_gbm)
final_rmse_gbm = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_gbm))
final_mse_gbm = mean_squared_error(y_test_rf_gbm, final_predictions_gbm)
final_mae_gbm = mean_absolute_error(y_test_rf_gbm, final_predictions_gbm)
final_r2_gbm = r2_score(y_test_rf_gbm, final_predictions_gbm)
print(f"Gradient Boosting - Training Time: {training_time_gbm:.2f}s, RMSE: {final_rmse_gbm:.4f}, MSE: {final_mse_gbm:.4f}, MAE: {final_mae_gbm:.4f}, R2: {final_r2_gbm:.4f}")

Best parameters for Gradient Boosting: {'n_estimators': 500, 'learning_rate': 0.05, 'max_depth': 5, 'loss': 'huber'}
Gradient Boosting - Training Time: 65.31s, RMSE: 0.3753, MSE: 0.1408, MAE: 0.2653, R2: 0.8386


In [14]:
# XGBoost
best_params_xgb = study_xgb.best_params
print("Best parameters for XGBoost:", best_params_xgb)

best_num_boost_round = best_params_xgb.pop('n_estimators')
start_time = time.time()
final_model_xgb = xgb.train(best_params_xgb, dtrain_val, num_boost_round=best_num_boost_round)
training_time_xgb = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_xgb, "./daegu_model/daegu_xgb_opt.pkl")

# 예측 (test 데이터로 예측)
final_predictions_xgb = final_model_xgb.predict(dtest)

# 평가 지표 계산
final_rmse_xgb = np.sqrt(mean_squared_error(dtest.get_label(), final_predictions_xgb))
final_mse_xgb = mean_squared_error(dtest.get_label(), final_predictions_xgb)
final_mae_xgb = mean_absolute_error(dtest.get_label(), final_predictions_xgb)
final_r2_xgb = r2_score(dtest.get_label(), final_predictions_xgb)

# 결과 출력
print(f"XGBoost - Training Time: {training_time_xgb:.2f}s, RMSE: {final_rmse_xgb:.4f}, MSE: {final_mse_xgb:.4f}, MAE: {final_mae_xgb:.4f}, R2: {final_r2_xgb:.4f}")

Best parameters for XGBoost: {'learning_rate': 0.01, 'max_depth': 8, 'subsample': 0.75, 'colsample_bytree': 0.5, 'booster': 'dart', 'n_estimators': 1000}
XGBoost - Training Time: 692.25s, RMSE: 0.3716, MSE: 0.1381, MAE: 0.2672, R2: 0.8418


In [15]:
# LightGBM
best_params_lgb = study_lgb.best_params
print("Best parameters for LightGBM:", best_params_lgb)

start_time = time.time()
final_model_lgb = lgb.train(best_params_lgb, ltrain_val, num_boost_round=best_params_lgb['n_estimators'])
training_time_lgb = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_lgb, "./daegu_model/daegu_lgb_opt.pkl")

# 예측 (test 데이터로 예측)
final_predictions_lgb = final_model_lgb.predict(X_test_rf_gbm)  # X_test_rf_gbm 사용

# 평가 지표 계산
final_rmse_lgb = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_lgb))
final_mse_lgb = mean_squared_error(y_test_rf_gbm, final_predictions_lgb)
final_mae_lgb = mean_absolute_error(y_test_rf_gbm, final_predictions_lgb)
final_r2_lgb = r2_score(y_test_rf_gbm, final_predictions_lgb)

# 결과 출력
print(f"LightGBM - Training Time: {training_time_lgb:.2f}s, RMSE: {final_rmse_lgb:.4f}, MSE: {final_mse_lgb:.4f}, MAE: {final_mae_lgb:.4f}, R2: {final_r2_lgb:.4f}")

Best parameters for LightGBM: {'learning_rate': 0.05, 'num_leaves': 64, 'subsample': 0.5, 'feature_fraction': 1.0, 'boosting_type': 'dart', 'n_estimators': 1000}


c:\Users\양태영\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.025641 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1890
[LightGBM] [Info] Number of data points in the train set: 32142, number of used features: 21
[LightGBM] [Info] Start training from score 1.313424
LightGBM - Training Time: 39.35s, RMSE: 0.3808, MSE: 0.1450, MAE: 0.2731, R2: 0.8339


In [16]:
# CatBoost
best_params_cat = study_cat.best_params
print("Best parameters for CatBoost:", best_params_cat)

start_time = time.time()
final_model_cat = cb.CatBoostRegressor(**best_params_cat, verbose=0, random_seed=SEED)
final_model_cat.fit(ctrain_val)
training_time_cat = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_cat, "./daegu_model/daegu_cat_opt.pkl")

final_predictions_cat = final_model_cat.predict(ctest)
final_rmse_cat = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_cat))
final_mse_cat = mean_squared_error(y_test_rf_gbm, final_predictions_cat)
final_mae_cat = mean_absolute_error(y_test_rf_gbm, final_predictions_cat)
final_r2_cat = r2_score(y_test_rf_gbm, final_predictions_cat)
print(f"CatBoost - Training Time: {training_time_cat:.2f}s, RMSE: {final_rmse_cat:.4f}, MSE: {final_mse_cat:.4f}, MAE: {final_mae_cat:.4f}, R2: {final_r2_cat:.4f}")

Best parameters for CatBoost: {'learning_rate': 0.03, 'depth': 10, 'l2_leaf_reg': 9}
CatBoost - Training Time: 44.24s, RMSE: 0.3695, MSE: 0.1366, MAE: 0.2634, R2: 0.8435


대전

In [12]:
# Random Forest
best_params_rf = study_rf.best_params
print("Best parameters for Random Forest:", best_params_rf)

# 모델 학습 및 저장
start_time = time.time()
final_model_rf = RandomForestRegressor(**best_params_rf, random_state=SEED)
final_model_rf.fit(X_train_val_rf_gbm, y_train_val_rf_gbm)
training_time_rf = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_rf, "./daejeon_model/daejeon_rf_opt.pkl")

# 모델 성능 평가
final_predictions_rf = final_model_rf.predict(X_test_rf_gbm)
final_rmse_rf = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_rf))
final_mse_rf = mean_squared_error(y_test_rf_gbm, final_predictions_rf)
final_mae_rf = mean_absolute_error(y_test_rf_gbm, final_predictions_rf)
final_r2_rf = r2_score(y_test_rf_gbm, final_predictions_rf)

print(f"Random Forest - Training Time: {training_time_rf:.2f}s, RMSE: {final_rmse_rf:.4f}, MSE: {final_mse_rf:.4f}, MAE: {final_mae_rf:.4f}, R2: {final_r2_rf:.4f}")

Best parameters for Random Forest: {'n_estimators': 128, 'max_features': 'log2'}
Random Forest - Training Time: 12.43s, RMSE: 0.4188, MSE: 0.1754, MAE: 0.3085, R2: 0.8297


In [13]:
# Gradient Boosting
best_params_gbm = study_gbm.best_params
print("Best parameters for Gradient Boosting:", best_params_gbm)

start_time = time.time()
final_model_gbm = GradientBoostingRegressor(**best_params_gbm)
final_model_gbm.fit(X_train_val_rf_gbm, y_train_val_rf_gbm)
training_time_gbm = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_gbm, "./daejeon_model/daejeon_gbm_opt.pkl")

final_predictions_gbm = final_model_gbm.predict(X_test_rf_gbm)
final_rmse_gbm = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_gbm))
final_mse_gbm = mean_squared_error(y_test_rf_gbm, final_predictions_gbm)
final_mae_gbm = mean_absolute_error(y_test_rf_gbm, final_predictions_gbm)
final_r2_gbm = r2_score(y_test_rf_gbm, final_predictions_gbm)
print(f"Gradient Boosting - Training Time: {training_time_gbm:.2f}s, RMSE: {final_rmse_gbm:.4f}, MSE: {final_mse_gbm:.4f}, MAE: {final_mae_gbm:.4f}, R2: {final_r2_gbm:.4f}")

Best parameters for Gradient Boosting: {'n_estimators': 500, 'learning_rate': 0.05, 'max_depth': 5, 'loss': 'huber'}
Gradient Boosting - Training Time: 73.20s, RMSE: 0.4045, MSE: 0.1636, MAE: 0.2908, R2: 0.8411


In [14]:
# XGBoost
best_params_xgb = study_xgb.best_params
print("Best parameters for XGBoost:", best_params_xgb)

best_num_boost_round = best_params_xgb.pop('n_estimators')
start_time = time.time()
final_model_xgb = xgb.train(best_params_xgb, dtrain_val, num_boost_round=best_num_boost_round)
training_time_xgb = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_xgb, "./daejeon_model/daejeon_xgb_opt.pkl")

# 예측 (test 데이터로 예측)
final_predictions_xgb = final_model_xgb.predict(dtest)

# 평가 지표 계산
final_rmse_xgb = np.sqrt(mean_squared_error(dtest.get_label(), final_predictions_xgb))
final_mse_xgb = mean_squared_error(dtest.get_label(), final_predictions_xgb)
final_mae_xgb = mean_absolute_error(dtest.get_label(), final_predictions_xgb)
final_r2_xgb = r2_score(dtest.get_label(), final_predictions_xgb)

# 결과 출력
print(f"XGBoost - Training Time: {training_time_xgb:.2f}s, RMSE: {final_rmse_xgb:.4f}, MSE: {final_mse_xgb:.4f}, MAE: {final_mae_xgb:.4f}, R2: {final_r2_xgb:.4f}")

Best parameters for XGBoost: {'learning_rate': 0.01, 'max_depth': 10, 'subsample': 0.75, 'colsample_bytree': 0.75, 'booster': 'gbtree', 'n_estimators': 1000}
XGBoost - Training Time: 20.06s, RMSE: 0.4010, MSE: 0.1608, MAE: 0.2903, R2: 0.8438


In [15]:
# LightGBM
best_params_lgb = study_lgb.best_params
print("Best parameters for LightGBM:", best_params_lgb)

start_time = time.time()
final_model_lgb = lgb.train(best_params_lgb, ltrain_val, num_boost_round=best_params_lgb['n_estimators'])
training_time_lgb = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_lgb, "./daejeon_model/daejeon_lgb_opt.pkl")

# 예측 (test 데이터로 예측)
final_predictions_lgb = final_model_lgb.predict(X_test_rf_gbm)  # X_test_rf_gbm 사용

# 평가 지표 계산
final_rmse_lgb = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_lgb))
final_mse_lgb = mean_squared_error(y_test_rf_gbm, final_predictions_lgb)
final_mae_lgb = mean_absolute_error(y_test_rf_gbm, final_predictions_lgb)
final_r2_lgb = r2_score(y_test_rf_gbm, final_predictions_lgb)

# 결과 출력
print(f"LightGBM - Training Time: {training_time_lgb:.2f}s, RMSE: {final_rmse_lgb:.4f}, MSE: {final_mse_lgb:.4f}, MAE: {final_mae_lgb:.4f}, R2: {final_r2_lgb:.4f}")

Best parameters for LightGBM: {'learning_rate': 0.01, 'num_leaves': 64, 'subsample': 0.5, 'feature_fraction': 1.0, 'boosting_type': 'gbdt', 'n_estimators': 1000}
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003142 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1677
[LightGBM] [Info] Number of data points in the train set: 32142, number of used features: 21
[LightGBM] [Info] Start training from score 1.356206


c:\Users\양태영\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


LightGBM - Training Time: 6.03s, RMSE: 0.4051, MSE: 0.1641, MAE: 0.2927, R2: 0.8406


In [16]:
# CatBoost
best_params_cat = study_cat.best_params
print("Best parameters for CatBoost:", best_params_cat)

start_time = time.time()
final_model_cat = cb.CatBoostRegressor(**best_params_cat, verbose=0, random_seed=SEED)
final_model_cat.fit(ctrain_val)
training_time_cat = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_cat, "./daejeon_model/daejeon_cat_opt.pkl")

final_predictions_cat = final_model_cat.predict(ctest)
final_rmse_cat = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_cat))
final_mse_cat = mean_squared_error(y_test_rf_gbm, final_predictions_cat)
final_mae_cat = mean_absolute_error(y_test_rf_gbm, final_predictions_cat)
final_r2_cat = r2_score(y_test_rf_gbm, final_predictions_cat)
print(f"CatBoost - Training Time: {training_time_cat:.2f}s, RMSE: {final_rmse_cat:.4f}, MSE: {final_mse_cat:.4f}, MAE: {final_mae_cat:.4f}, R2: {final_r2_cat:.4f}")

Best parameters for CatBoost: {'learning_rate': 0.03, 'depth': 10, 'l2_leaf_reg': 7}
CatBoost - Training Time: 39.01s, RMSE: 0.4016, MSE: 0.1613, MAE: 0.2904, R2: 0.8434


인천

In [12]:
# Random Forest
best_params_rf = study_rf.best_params
print("Best parameters for Random Forest:", best_params_rf)

# 모델 학습 및 저장
start_time = time.time()
final_model_rf = RandomForestRegressor(**best_params_rf, random_state=SEED)
final_model_rf.fit(X_train_val_rf_gbm, y_train_val_rf_gbm)
training_time_rf = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_rf, "./incheon_model/incheon_rf_opt.pkl")

# 모델 성능 평가
final_predictions_rf = final_model_rf.predict(X_test_rf_gbm)
final_rmse_rf = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_rf))
final_mse_rf = mean_squared_error(y_test_rf_gbm, final_predictions_rf)
final_mae_rf = mean_absolute_error(y_test_rf_gbm, final_predictions_rf)
final_r2_rf = r2_score(y_test_rf_gbm, final_predictions_rf)

print(f"Random Forest - Training Time: {training_time_rf:.2f}s, RMSE: {final_rmse_rf:.4f}, MSE: {final_mse_rf:.4f}, MAE: {final_mae_rf:.4f}, R2: {final_r2_rf:.4f}")

Best parameters for Random Forest: {'n_estimators': 128, 'max_features': 'log2'}
Random Forest - Training Time: 12.64s, RMSE: 0.4243, MSE: 0.1801, MAE: 0.3198, R2: 0.7893


In [13]:
# Gradient Boosting
best_params_gbm = study_gbm.best_params
print("Best parameters for Gradient Boosting:", best_params_gbm)

start_time = time.time()
final_model_gbm = GradientBoostingRegressor(**best_params_gbm)
final_model_gbm.fit(X_train_val_rf_gbm, y_train_val_rf_gbm)
training_time_gbm = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_gbm, "./incheon_model/incheon_gbm_opt.pkl")

final_predictions_gbm = final_model_gbm.predict(X_test_rf_gbm)
final_rmse_gbm = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_gbm))
final_mse_gbm = mean_squared_error(y_test_rf_gbm, final_predictions_gbm)
final_mae_gbm = mean_absolute_error(y_test_rf_gbm, final_predictions_gbm)
final_r2_gbm = r2_score(y_test_rf_gbm, final_predictions_gbm)
print(f"Gradient Boosting - Training Time: {training_time_gbm:.2f}s, RMSE: {final_rmse_gbm:.4f}, MSE: {final_mse_gbm:.4f}, MAE: {final_mae_gbm:.4f}, R2: {final_r2_gbm:.4f}")

Best parameters for Gradient Boosting: {'n_estimators': 500, 'learning_rate': 0.05, 'max_depth': 5, 'loss': 'huber'}
Gradient Boosting - Training Time: 63.87s, RMSE: 0.4048, MSE: 0.1639, MAE: 0.2957, R2: 0.8082


In [14]:
# XGBoost
best_params_xgb = study_xgb.best_params
print("Best parameters for XGBoost:", best_params_xgb)

best_num_boost_round = best_params_xgb.pop('n_estimators')
start_time = time.time()
final_model_xgb = xgb.train(best_params_xgb, dtrain_val, num_boost_round=best_num_boost_round)
training_time_xgb = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_xgb, "./incheon_model/incheon_xgb_opt.pkl")

# 예측 (test 데이터로 예측)
final_predictions_xgb = final_model_xgb.predict(dtest)

# 평가 지표 계산
final_rmse_xgb = np.sqrt(mean_squared_error(dtest.get_label(), final_predictions_xgb))
final_mse_xgb = mean_squared_error(dtest.get_label(), final_predictions_xgb)
final_mae_xgb = mean_absolute_error(dtest.get_label(), final_predictions_xgb)
final_r2_xgb = r2_score(dtest.get_label(), final_predictions_xgb)

# 결과 출력
print(f"XGBoost - Training Time: {training_time_xgb:.2f}s, RMSE: {final_rmse_xgb:.4f}, MSE: {final_mse_xgb:.4f}, MAE: {final_mae_xgb:.4f}, R2: {final_r2_xgb:.4f}")

Best parameters for XGBoost: {'learning_rate': 0.01, 'max_depth': 8, 'subsample': 0.75, 'colsample_bytree': 0.75, 'booster': 'gbtree', 'n_estimators': 1000}
XGBoost - Training Time: 9.71s, RMSE: 0.3978, MSE: 0.1583, MAE: 0.2942, R2: 0.8148


In [15]:
# LightGBM
best_params_lgb = study_lgb.best_params
print("Best parameters for LightGBM:", best_params_lgb)

start_time = time.time()
final_model_lgb = lgb.train(best_params_lgb, ltrain_val, num_boost_round=best_params_lgb['n_estimators'])
training_time_lgb = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_lgb, "./incheon_model/incheon_lgb_opt.pkl")

# 예측 (test 데이터로 예측)
final_predictions_lgb = final_model_lgb.predict(X_test_rf_gbm)  # X_test_rf_gbm 사용

# 평가 지표 계산
final_rmse_lgb = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_lgb))
final_mse_lgb = mean_squared_error(y_test_rf_gbm, final_predictions_lgb)
final_mae_lgb = mean_absolute_error(y_test_rf_gbm, final_predictions_lgb)
final_r2_lgb = r2_score(y_test_rf_gbm, final_predictions_lgb)

# 결과 출력
print(f"LightGBM - Training Time: {training_time_lgb:.2f}s, RMSE: {final_rmse_lgb:.4f}, MSE: {final_mse_lgb:.4f}, MAE: {final_mae_lgb:.4f}, R2: {final_r2_lgb:.4f}")

Best parameters for LightGBM: {'learning_rate': 0.05, 'num_leaves': 64, 'subsample': 0.5, 'feature_fraction': 1.0, 'boosting_type': 'dart', 'n_estimators': 1000}
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003431 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1803
[LightGBM] [Info] Number of data points in the train set: 32142, number of used features: 21
[LightGBM] [Info] Start training from score 1.143254


c:\Users\양태영\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


LightGBM - Training Time: 38.55s, RMSE: 0.4042, MSE: 0.1633, MAE: 0.3010, R2: 0.8089


In [16]:
# CatBoost
best_params_cat = study_cat.best_params
print("Best parameters for CatBoost:", best_params_cat)

start_time = time.time()
final_model_cat = cb.CatBoostRegressor(**best_params_cat, verbose=0, random_seed=SEED)
final_model_cat.fit(ctrain_val)
training_time_cat = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_cat, "./incheon_model/incheon_cat_opt.pkl")

final_predictions_cat = final_model_cat.predict(ctest)
final_rmse_cat = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_cat))
final_mse_cat = mean_squared_error(y_test_rf_gbm, final_predictions_cat)
final_mae_cat = mean_absolute_error(y_test_rf_gbm, final_predictions_cat)
final_r2_cat = r2_score(y_test_rf_gbm, final_predictions_cat)
print(f"CatBoost - Training Time: {training_time_cat:.2f}s, RMSE: {final_rmse_cat:.4f}, MSE: {final_mse_cat:.4f}, MAE: {final_mae_cat:.4f}, R2: {final_r2_cat:.4f}")

Best parameters for CatBoost: {'learning_rate': 0.03, 'depth': 10, 'l2_leaf_reg': 9}
CatBoost - Training Time: 41.99s, RMSE: 0.3953, MSE: 0.1562, MAE: 0.2914, R2: 0.8172


서울

In [12]:
# Random Forest
best_params_rf = study_rf.best_params
print("Best parameters for Random Forest:", best_params_rf)

# 모델 학습 및 저장
start_time = time.time()
final_model_rf = RandomForestRegressor(**best_params_rf, random_state=SEED)
final_model_rf.fit(X_train_val_rf_gbm, y_train_val_rf_gbm)
training_time_rf = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_rf, "./seoul_model/seoul_rf_opt.pkl")

# 모델 성능 평가
final_predictions_rf = final_model_rf.predict(X_test_rf_gbm)
final_rmse_rf = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_rf))
final_mse_rf = mean_squared_error(y_test_rf_gbm, final_predictions_rf)
final_mae_rf = mean_absolute_error(y_test_rf_gbm, final_predictions_rf)
final_r2_rf = r2_score(y_test_rf_gbm, final_predictions_rf)

print(f"Random Forest - Training Time: {training_time_rf:.2f}s, RMSE: {final_rmse_rf:.4f}, MSE: {final_mse_rf:.4f}, MAE: {final_mae_rf:.4f}, R2: {final_r2_rf:.4f}")

Best parameters for Random Forest: {'n_estimators': 128, 'max_features': 'log2'}
Random Forest - Training Time: 14.46s, RMSE: 0.4369, MSE: 0.1909, MAE: 0.3247, R2: 0.7745


In [13]:
# Gradient Boosting
best_params_gbm = study_gbm.best_params
print("Best parameters for Gradient Boosting:", best_params_gbm)

start_time = time.time()
final_model_gbm = GradientBoostingRegressor(**best_params_gbm)
final_model_gbm.fit(X_train_val_rf_gbm, y_train_val_rf_gbm)
training_time_gbm = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_gbm, "./seoul_model/seoul_gbm_opt.pkl")

final_predictions_gbm = final_model_gbm.predict(X_test_rf_gbm)
final_rmse_gbm = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_gbm))
final_mse_gbm = mean_squared_error(y_test_rf_gbm, final_predictions_gbm)
final_mae_gbm = mean_absolute_error(y_test_rf_gbm, final_predictions_gbm)
final_r2_gbm = r2_score(y_test_rf_gbm, final_predictions_gbm)
print(f"Gradient Boosting - Training Time: {training_time_gbm:.2f}s, RMSE: {final_rmse_gbm:.4f}, MSE: {final_mse_gbm:.4f}, MAE: {final_mae_gbm:.4f}, R2: {final_r2_gbm:.4f}")

Best parameters for Gradient Boosting: {'n_estimators': 500, 'learning_rate': 0.05, 'max_depth': 5, 'loss': 'huber'}
Gradient Boosting - Training Time: 73.22s, RMSE: 0.4239, MSE: 0.1797, MAE: 0.3062, R2: 0.7877


In [14]:
# XGBoost
best_params_xgb = study_xgb.best_params
print("Best parameters for XGBoost:", best_params_xgb)

best_num_boost_round = best_params_xgb.pop('n_estimators')
start_time = time.time()
final_model_xgb = xgb.train(best_params_xgb, dtrain_val, num_boost_round=best_num_boost_round)
training_time_xgb = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_xgb, "./seoul_model/seoul_xgb_opt.pkl")

# 예측 (test 데이터로 예측)
final_predictions_xgb = final_model_xgb.predict(dtest)

# 평가 지표 계산
final_rmse_xgb = np.sqrt(mean_squared_error(dtest.get_label(), final_predictions_xgb))
final_mse_xgb = mean_squared_error(dtest.get_label(), final_predictions_xgb)
final_mae_xgb = mean_absolute_error(dtest.get_label(), final_predictions_xgb)
final_r2_xgb = r2_score(dtest.get_label(), final_predictions_xgb)

# 결과 출력
print(f"XGBoost - Training Time: {training_time_xgb:.2f}s, RMSE: {final_rmse_xgb:.4f}, MSE: {final_mse_xgb:.4f}, MAE: {final_mae_xgb:.4f}, R2: {final_r2_xgb:.4f}")

Best parameters for XGBoost: {'learning_rate': 0.05, 'max_depth': 6, 'subsample': 0.75, 'colsample_bytree': 1.0, 'booster': 'dart', 'n_estimators': 250}
XGBoost - Training Time: 47.85s, RMSE: 0.4234, MSE: 0.1793, MAE: 0.3087, R2: 0.7881


In [15]:
# LightGBM
best_params_lgb = study_lgb.best_params
print("Best parameters for LightGBM:", best_params_lgb)

start_time = time.time()
final_model_lgb = lgb.train(best_params_lgb, ltrain_val, num_boost_round=best_params_lgb['n_estimators'])
training_time_lgb = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_lgb, "./seoul_model/seoul_lgb_opt.pkl")

# 예측 (test 데이터로 예측)
final_predictions_lgb = final_model_lgb.predict(X_test_rf_gbm)  # X_test_rf_gbm 사용

# 평가 지표 계산
final_rmse_lgb = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_lgb))
final_mse_lgb = mean_squared_error(y_test_rf_gbm, final_predictions_lgb)
final_mae_lgb = mean_absolute_error(y_test_rf_gbm, final_predictions_lgb)
final_r2_lgb = r2_score(y_test_rf_gbm, final_predictions_lgb)

# 결과 출력
print(f"LightGBM - Training Time: {training_time_lgb:.2f}s, RMSE: {final_rmse_lgb:.4f}, MSE: {final_mse_lgb:.4f}, MAE: {final_mae_lgb:.4f}, R2: {final_r2_lgb:.4f}")

Best parameters for LightGBM: {'learning_rate': 0.01, 'num_leaves': 64, 'subsample': 0.5, 'feature_fraction': 1.0, 'boosting_type': 'gbdt', 'n_estimators': 1000}
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003680 seconds.
You can set `force_col_wise=true` to remove the overhead.

c:\Users\양태영\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")



[LightGBM] [Info] Total Bins 1897
[LightGBM] [Info] Number of data points in the train set: 32142, number of used features: 21
[LightGBM] [Info] Start training from score 1.115901
LightGBM - Training Time: 6.20s, RMSE: 0.4218, MSE: 0.1779, MAE: 0.3062, R2: 0.7897


In [16]:
# CatBoost
best_params_cat = study_cat.best_params
print("Best parameters for CatBoost:", best_params_cat)

start_time = time.time()
final_model_cat = cb.CatBoostRegressor(**best_params_cat, verbose=0, random_seed=SEED)
final_model_cat.fit(ctrain_val)
training_time_cat = time.time() - start_time

# 학습된 모델 저장
joblib.dump(final_model_cat, "./seoul_model/seoul_cat_opt.pkl")

final_predictions_cat = final_model_cat.predict(ctest)
final_rmse_cat = np.sqrt(mean_squared_error(y_test_rf_gbm, final_predictions_cat))
final_mse_cat = mean_squared_error(y_test_rf_gbm, final_predictions_cat)
final_mae_cat = mean_absolute_error(y_test_rf_gbm, final_predictions_cat)
final_r2_cat = r2_score(y_test_rf_gbm, final_predictions_cat)
print(f"CatBoost - Training Time: {training_time_cat:.2f}s, RMSE: {final_rmse_cat:.4f}, MSE: {final_mse_cat:.4f}, MAE: {final_mae_cat:.4f}, R2: {final_r2_cat:.4f}")

Best parameters for CatBoost: {'learning_rate': 0.03, 'depth': 10, 'l2_leaf_reg': 9}
CatBoost - Training Time: 44.73s, RMSE: 0.4175, MSE: 0.1743, MAE: 0.3056, R2: 0.7940
